# Tutorial: Train a diabetes prediction model and create a batch inferencing pipeline. 
[This dataset](https://github.com/maluvinita/Tutorials/tree/main/data) is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective is to predict based on diagnostic measurements whether a patient has diabetes.

This tutorial shows how we can use a trained model for prediction, if we have stored our data in different files.  

This tutorial includes following:
- Train the model using diabetes dataset & register it.
- Create a batch inferencing pipeline. 

## Importing AML packages

In [ ]:
import sys
import os
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

print(sys.version)
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Create workspace
If the workspace already exists connect to it

In [ ]:
ws = Workspace.create(
    name = "Your Workspace Name",
    subscription_id = "Your Subsription Id",
    resource_group = "Your Resource Group", 
    location = "Your location",  # e.g "westus"
    exist_ok = True,
    show_output = True)

ws.write_config()

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

_We already learned how to register a data set in our workspace in [this tutorial ](https://github.com/maluvinita/Tutorials/blob/main/Deploy-Real-Time-Inferencing-Service/diabetes-prediction-model-AML.ipynb). Therfore we directly use that registered dataset in this tutorial to train the model. _

## Train a model from a file dataset
Run the following two code cells to create:

- A folder named diabetes_training
- A script that trains a classification model by using a file dataset that is passed to is as an input.

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import os
import argparse
from azureml.core import Dataset, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# Get script arguments (rgularization rate and file dataset mount point)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--input-data', type=str, dest='dataset_folder', help='data mount point')
args = parser.parse_args()

# Set regularization hyperparameter (passed as an argument to the script)
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data_path = args.dataset_folder # Get the training data path from the input
# (You could also just use args.dataset_folder if you don't want to rely on a hard-coded friendly name)

# Read the files
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

model_file = 'diabetes_pipeline_model.pkl'
os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/'+model_file)

run.complete()


## Run the training script as an experiment
The conda environment is built on-demand the first time the experiment is run, and cached for future runs that use the same configuration; so the first run will take a little longer.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails
import shutil

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes file")

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("pipeline_exp_environment", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_download(path_on_compute="/tmp/training_files")], # Reference to dataset location
                                environment=env, # Use the environment created previously
                                )


#Create an experiment to track the runs in your workspace
experiment_name = 'predict-diabetes-pipeline'
experiment = Experiment(workspace=ws, name=experiment_name)

# remove the existing file becuse it doesn't overwrite the file 
import shutil
download_file_path = "/tmp/training_files"
if os.path.exists(download_file_path):
    shutil.rmtree(download_file_path)

# submit the experiment
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

## Register the trained model
Note that the outputs of the experiment include the trained model file (**diabetes_pipeline_model.pkl**). We can register this model in your AML workspace, making it possible to track model versions and retrieve them later.

In [ ]:
# Register the model
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_pipeline_model.pkl', model_name='diabetes_pipeline_model',
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## Download the model in current working directory.
We need this model while we create a pipeline. 

In [ ]:
model_obj = Model(ws, 'diabetes_pipeline_model')
model_path = model_obj.download(exist_ok = True)

## Generate batch data

We will generate a random sample from our diabetes CSV file, upload that data to a datastore in the Azure Machine Learning workspace, and register a dataset for it.

In [ ]:
import pandas as pd
import os

# Set default data store
ws.set_default_datastore('workspaceblobstore')
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

# Load the diabetes data
diabetes = pd.read_csv('diabetes-data/diabetes.csv')
# Get a 100-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=100).values

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")


## Upload batch data
Upload the data in the workspaceblob store.

_Note : please remove all the hidden files eg .amlignore etc. if those were created during generation of batch data _   

In [ ]:
# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-dataset", overwrite=True, show_progress=True)

print("Done!")

## Register a dataset for the batch input data

In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-dataset/'))

try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-dataset',
                                             description='batch-data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)


## Create a pipeline for batch inferencing

Now we're ready to define the pipeline we'll use for batch inferencing. Our pipeline will need Python code to perform the batch inferencing, so let's create a folder where we can keep all the files used by the pipeline

In [ ]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

Now we'll create a Python script to do the actual work, and save it in the pipeline folder:

In [ ]:
%%writefile $experiment_folder/batch_diabetes.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    global model
    # Get the path to the deployed model file and load it   
    model_path = Model.get_model_path('diabetes_pipeline_model')
    model = joblib.load(model_path)
   

def run(mini_batch):
    # This runs for each batch
    resultList = []
    # process each file in the batch
    for f in mini_batch:
        # Read the comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for prediction (model expects multiple items)
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

## Create compute
We'll need a compute context for the pipeline, so we'll use the following code to specify an Azure Machine Learning compute cluster (it will be created if it doesn't already exist).

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your cluster name"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

The pipeline will need an environment in which to run, so we'll create a Conda specification that includes the packages that the code uses.

_Note : Use Python > 3.6, Joblib latest version with python 3.6 gives an error while you load a model. _

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("experiment_batch_env", "environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

We are going to use a pipeline to run the batch prediction script, generate predictions from the input data, and save the results as a text file in the output folder. To do this, we can use a ParallelRunStep, which enables the batch data to be processed in parallel and the results collated in a single output file named _parallel_run_step.txt._

[OutputFileDatasetConfig](https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.data.outputfiledatasetconfig?view=azure-ml-py)

In [ ]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig

output_dir = OutputFileDatasetConfig(name='inferences')

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="batch_diabetes.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-diabetes',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('diabetes_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

## Put the step into a pipeline, and run it.

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'predict-diabetes-pipeline').submit(pipeline)
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

When the pipeline has finished running, the resulting predictions will have been saved in the outputs of the experiment associated with the first (and only) step in the pipeline. We can retrieve it as follows:
```

I've tried to retrived the output but found there was a [BUG](https://github.com/Azure/azure-sdk-for-python/issues/23784) and it didn't get resolved while I was working. So I'm writing all the steps here to retrieve the output. Hopefully when you work on this tutorial it will get resolved.
```

In [ ]:
import pandas as pd
import shutil
from azureml.pipeline.core import PipelineRun, StepRun, PortDataReference

# Remove the local results folder if left over from a previous run
shutil.rmtree('diabetes-results', ignore_errors=True)

# Get the run for the first step and download its output
step_run = next(pipeline_run.get_steps())
prediction_output = step_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

## Publish the Pipeline and use its REST Interface

Now that we have a working pipeline for batch inferencing, we can publish it and use a REST endpoint to run it from an application.

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name='diabetes-batch-pipeline', description='Batch scoring of diabetes data', version='1.0')

published_pipeline

Note that the published pipeline has an endpoint, which we can see in the Azure portal. We can also find it as a property of the published pipeline object:

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. To test this out, we'll use the authorization header from our current connection to our Azure workspace, which we can get using the following code:

> **Note**: A real application would require a service principal with which to be authenticated.

In [22]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

Authentication header ready.


Now we're ready to call the REST interface. The pipeline runs asynchronously, so we'll get an identifier back, which we can use to track the pipeline experiment as it runs:

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "predict-diabetes-pipeline"})
run_id = response.json()["Id"]
run_id

Since we have the run ID, we can use the **RunDetails** widget to view the experiment as it runs:

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments['predict-diabetes-pipeline'], run_id)

# Block until the run completes
published_pipeline_run.wait_for_completion(show_output=True)

Wait for the pipeline run to complete. As before, the results are in the output of the first pipeline step. Rewrite the code to see the output

# Summary

Here we learn how to create a batch inference pipeline to predict the result. And we learn how to publish the pipeline and use its endpoint for a prediction application.